In [ ]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get all stock info
msft.info

In [ ]:
hist = msft.history(period="1mo")
print(hist)

In [ ]:
msft.history_metadata

In [ ]:
msft.quarterly_income_stmt

In [ ]:
msft.earnings_estimate
msft.quarterly_cashflow

In [ ]:
import yfinance as yf

# 设置股票代码
ticker = 'AAPL'  # 例如：苹果公司股票代码

# 下载股票数据
stock = yf.Ticker(ticker)

# 获取财务报表数据
balance_sheet = stock.balance_sheet     # 资产负债表 (年度)
income_statement = stock.financials     # 利润表 (年度)
cashflow = stock.cashflow               # 现金流量表 (年度)

# 获取季度财务报表数据
quarterly_balance_sheet = stock.quarterly_balance_sheet  # 资产负债表 (季度)
quarterly_income_statement = stock.quarterly_financials  # 利润表 (季度)
quarterly_cashflow = stock.quarterly_cashflow            # 现金流量表 (季度)

# 打印数据
print("年度资产负债表：\n", balance_sheet)
print("季度资产负债表：\n", quarterly_balance_sheet)

# 获取某个年份的财务数据，例如 2022 年的资产负债表
balance_sheet_2022 = balance_sheet.loc[:, balance_sheet.columns.year == 2022]
print("2022 年资产负债表：\n", balance_sheet_2022)



In [ ]:
import yfinance as yf

# 假设你感兴趣的是技术行业的ETF，比如XLK（科技板块）
etf_ticker = 'XLK'  # 这是一个科技行业相关的ETF

# 下载ETF数据
etf = yf.Ticker(etf_ticker)

# 获取成分股
print(etf)

etf.info

# 打印成分股
# print(holdings)


In [ ]:
from datetime import datetime, timedelta
import numpy as np

def get_ticker_market_cap(ticker, date):
    try:
        ticker_obj = yf.Ticker(ticker)
        date_obj = datetime.strptime(date, '%Y-%m-%d')
        new_date_obj = date_obj + timedelta(days=10)
        end_date = new_date_obj.strftime('%Y-%m-%d')
        historical_data = ticker_obj.history(start=date, end=end_date)
        shares_outstanding = ticker_obj.info['sharesOutstanding']
        return float(historical_data['Close'].iloc[0] * shares_outstanding / 1000000)
    except:
        return np.nan


print(get_ticker_market_cap("AAPL","2023-9-21"))
# ticker_obj.info['sharesOutstanding']

# # 获取股票的流通股数量 (Shares Outstanding)
# shares_outstanding = ticker_obj.info['sharesOutstanding']

# # 计算市值 = 收盘价 * 流通股数量
# market_cap = historical_data['Close'].iloc[0] * shares_outstanding

# # 打印结果
# print(f"2023-10-01 的市值为: {market_cap}")

In [ ]:
def get_finace_df(ticker):
    try:
        stock = yf.Ticker(ticker)

        # 获取季度财务报表
        balance_sheet_quarterly = stock.quarterly_balance_sheet  # 季度资产负债表
        income_statement_quarterly = stock.quarterly_financials  # 季度利润表
        cashflow_quarterly = stock.quarterly_cashflow            # 季度现金流量表

        def handle(df): # 转置，删除最后一行
            df = df.transpose()
            df = df.iloc[:min(5,df.shape[0])]
            return df
        balance_sheet_quarterly = handle(balance_sheet_quarterly)
        income_statement_quarterly = handle(income_statement_quarterly)
        cashflow_quarterly = handle(cashflow_quarterly)

        # 将3个df列合并
        df_merged = pd.merge(balance_sheet_quarterly, income_statement_quarterly, left_index=True, right_index=True)
        df_merged = pd.merge(df_merged, cashflow_quarterly, left_index=True, right_index=True)
        # 插入Ticker和Sector
        df_merged.insert(0, 'Ticker', [ticker]*df_merged.shape[0])
        df_merged = df_merged.reset_index()
        df_merged.rename(columns={'index': 'Quarter'}, inplace=True)
        # 插入市值（Y）
        quarter_list = list(df_merged.Quarter)
        marker_cap_list = []
        for quarter in quarter_list:
            quarter = quarter.strftime('%Y-%m-%d')
            marker_cap = get_ticker_market_cap(ticker, quarter)
            if marker_cap == -1:
                return -1
            else:
                marker_cap_list.append(marker_cap)
        df_merged.insert(df_merged.shape[1], 'Market Cap(M)', marker_cap_list)
        if df_merged.shape[0] == 0:
            return -1
        print(df_merged.shape)
        return df_merged
    except:
        return -1

get_finace_df("DXLG")

Sectors:
['Healthcare' 'Basic Materials' 'Financial' 'Consumer Defensive'
 'Industrials' 'Technology' 'Consumer Cyclical' 'Real Estate'
 'Communication Services' 'Energy' 'Utilities']

In [ ]:
not isinstance(-1, float)

In [6]:
import pandas as pd

FEATURE_PATH = "importance/"
df_features = pd.read_csv(FEATURE_PATH + "Utilities" + "_feature_importance.csv")
df_features.columns.values[0] = 'Feature'
top_20 = list(df_features.sort_values(by='Importance', ascending=False).head(20)["Feature"])
top_20

['Common Stock Dividend Paid',
 'Total Revenue',
 'Operating Cash Flow',
 'Accumulated Depreciation',
 'Free Cash Flow',
 'Capital Expenditure Reported',
 'Other Investments',
 'Goodwill',
 'Non Current Deferred Taxes Liabilities',
 'Current Provisions',
 'Interest Payable',
 'Depreciation And Amortization In Income Statement',
 'Other Properties',
 'Other Intangible Assets',
 'Total Capitalization',
 'Other Current Liabilities',
 'Selling General And Administration',
 'Minority Interest',
 'Invested Capital',
 'Other Receivables']